In [1]:
from serpapi import GoogleSearch
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import os
from functions import *

In [2]:
coin = 'ETH'
queries = None
with open('queries.txt') as queries:
    query = queries.read().splitlines()
    query = [f'{coin} {q}' for q in query]
    queries = query

In [3]:
PATH = f'newspapers/{coin}_newspapers.csv'
newspapers = pd.DataFrame()
if(os.path.exists(PATH)):
    newspapers = pd.read_csv(PATH)

for q in tqdm(queries):
    stuff = get_newspapers(q)
    newspapers = pd.concat([newspapers, stuff], ignore_index=True)

newspapers.to_csv(PATH)

100%|██████████| 3/3 [23:54<00:00, 478.19s/it]
